# Pilot Scheduling
## Using GEKKO with binary constraints to solve a pilot scheduling problem

A pilot can be in 2 states:
* Off - not flying
* On - flying

Pilots are either available or not available for each go.

In [1]:
import pandas as pd
from gekko import GEKKO
import math

In [2]:
pilots = pd.read_csv('csv/pilot_availability_real.csv',index_col=['PILOT'])
pilots

,1,2,3,4,5,6,7,8,9
PILOT,,,,,,,,,
WORM,0,0,0,0,0,0,0,0,0
SKIDS,0,0,0,0,0,0,0,0,0
SHINER,0,0,0,0,0,0,0,0,0
APE,0,0,0,0,0,0,0,0,0
BUSTER,0,0,0,0,0,0,0,0,0
HATTRICK,0,0,0,0,0,0,0,0,0
JACKAL,0,0,0,0,0,0,0,0,0
HOOPS,0,0,0,0,0,0,0,0,0
IRISH,0,0,0,0,0,0,0,0,0


In [3]:
pilot_quals = pd.read_csv('csv/pilot_qual_real.csv',index_col=['PILOT'])
pilot_quals

,QUAL
PILOT,
WORM,4
SKIDS,4
SHINER,4
APE,0
BUSTER,2
HATTRICK,2
JACKAL,3
HOOPS,2
IRISH,1


We'll also import our demand data

In [4]:
lines = pd.read_csv('csv/go_demand_real.csv', index_col=['GO','Line'])
lines

Requirement
GO Line             
1  101             2
   102             1
   103             2
   104             1
   105             2
...              ...
9  904             1
   905             2
   906             1
   907             2
   908             1

[64 rows x 1 columns]

Pilot status is modelled as a binary variable. It will have a value of 1 if the pilot is flying and a value of 0 when the pilot is off.

Binary variables are the same as integer variables but constrained to be >= 0 and <=1

Again this has a value for each month for each factory, again given by the index of our DataFrame

In [5]:
for go,line in lines.index:
    for pilot in pilots.index:
        if(pilots.loc[pilot][go-1]):
            print("%s is unavailable for %d"%(pilot,line))

In [6]:
m=GEKKO()

In [7]:
pilot_status={}

In [8]:
for Line  in lines.index:
    for PILOT in pilots.index:
        pilot_status[Line,PILOT] = m.Var(lb=0,ub=1)

In [9]:
PQ={}
for pilot in pilots.index:
    PQ[pilot]=m.Param(value=pilot_quals.loc[pilot,'QUAL']) #Pilot quals in GEKKO parameter

In [10]:
LR={}
for line in lines.index:
    LR[line]=m.Param(value=lines.loc[line, 'Requirement']) #Line Requirements in GEKKO parameter

In [11]:
pilot_status

{((1, 101), 'WORM'): 0,
 ((1, 101), 'SKIDS'): 0,
 ((1, 101), 'SHINER'): 0,
 ((1, 101), 'APE'): 0,
 ((1, 101), 'BUSTER'): 0,
 ((1, 101), 'HATTRICK'): 0,
 ((1, 101), 'JACKAL'): 0,
 ((1, 101), 'HOOPS'): 0,
 ((1, 101), 'IRISH'): 0,
 ((1, 101), 'CASS'): 0,
 ((1, 101), 'SHADOW'): 0,
 ((1, 101), 'BOND'): 0,
 ((1, 101), 'NAPE'): 0,
 ((1, 101), 'AMP'): 0,
 ((1, 101), 'FACE'): 0,
 ((1, 101), 'DEFOORE'): 0,
 ((1, 101), 'DURNIN'): 0,
 ((1, 101), 'CLIP'): 0,
 ((1, 101), 'COPEN'): 0,
 ((1, 101), 'CRUTCH'): 0,
 ((1, 101), 'REBEL'): 0,
 ((1, 101), 'TAOS'): 0,
 ((1, 101), 'COACH'): 0,
 ((1, 101), 'HEIST'): 0,
 ((1, 101), 'HOLSTER'): 0,
 ((1, 101), 'KONG'): 0,
 ((1, 101), 'LITTLE'): 0,
 ((1, 101), 'DUKE'): 0,
 ((1, 101), 'PITBULL'): 0,
 ((1, 101), 'HEAT'): 0,
 ((1, 101), 'FLIP'): 0,
 ((1, 102), 'WORM'): 0,
 ((1, 102), 'SKIDS'): 0,
 ((1, 102), 'SHINER'): 0,
 ((1, 102), 'APE'): 0,
 ((1, 102), 'BUSTER'): 0,
 ((1, 102), 'HATTRICK'): 0,
 ((1, 102), 'JACKAL'): 0,
 ((1, 102), 'HOOPS'): 0,
 ((1, 102), 'IRISH'):

We instantiate our model and use GEKKO solver.
The aim is always to minimise the objective function with the GEKKO solver.

In [12]:
m.options.SOLVER=1

In [13]:
m.Obj( 1*sum(pilot_status[line,pilot] for line in lines.index) for pilot in pilots.index)
#     [sum(sum((1/(abs(go2-go1)+1))*
#              pilot_status[(go1,line1),pilot]*
#              LR[go1,line1]*
#              pilot_status[((go2,line2),pilot)]*
#              LR[go2,line2] for go1,line1 in lines.index for go2,line2 in lines.index if go1 )
#          for pilot in pilots.index
#         )])

In [14]:
for Line  in lines.index:
    for PILOT in pilots.index:
        print(Line)
        print(PILOT)
        #sum(pilot_status[Line,PILOT])

(1, 101)
WORM
(1, 101)
SKIDS
(1, 101)
SHINER
(1, 101)
APE
(1, 101)
BUSTER
(1, 101)
HATTRICK
(1, 101)
JACKAL
(1, 101)
HOOPS
(1, 101)
IRISH
(1, 101)
CASS
(1, 101)
SHADOW
(1, 101)
BOND
(1, 101)
NAPE
(1, 101)
AMP
(1, 101)
FACE
(1, 101)
DEFOORE
(1, 101)
DURNIN
(1, 101)
CLIP
(1, 101)
COPEN
(1, 101)
CRUTCH
(1, 101)
REBEL
(1, 101)
TAOS
(1, 101)
COACH
(1, 101)
HEIST
(1, 101)
HOLSTER
(1, 101)
KONG
(1, 101)
LITTLE
(1, 101)
DUKE
(1, 101)
PITBULL
(1, 101)
HEAT
(1, 101)
FLIP
(1, 102)
WORM
(1, 102)
SKIDS
(1, 102)
SHINER
(1, 102)
APE
(1, 102)
BUSTER
(1, 102)
HATTRICK
(1, 102)
JACKAL
(1, 102)
HOOPS
(1, 102)
IRISH
(1, 102)
CASS
(1, 102)
SHADOW
(1, 102)
BOND
(1, 102)
NAPE
(1, 102)
AMP
(1, 102)
FACE
(1, 102)
DEFOORE
(1, 102)
DURNIN
(1, 102)
CLIP
(1, 102)
COPEN
(1, 102)
CRUTCH
(1, 102)
REBEL
(1, 102)
TAOS
(1, 102)
COACH
(1, 102)
HEIST
(1, 102)
HOLSTER
(1, 102)
KONG
(1, 102)
LITTLE
(1, 102)
DUKE
(1, 102)
PITBULL
(1, 102)
HEAT
(1, 102)
FLIP
(1, 103)
WORM
(1, 103)
SKIDS
(1, 103)
SHINER
(1, 103)
APE
(1, 103)
B

TAOS
(4, 401)
COACH
(4, 401)
HEIST
(4, 401)
HOLSTER
(4, 401)
KONG
(4, 401)
LITTLE
(4, 401)
DUKE
(4, 401)
PITBULL
(4, 401)
HEAT
(4, 401)
FLIP
(4, 402)
WORM
(4, 402)
SKIDS
(4, 402)
SHINER
(4, 402)
APE
(4, 402)
BUSTER
(4, 402)
HATTRICK
(4, 402)
JACKAL
(4, 402)
HOOPS
(4, 402)
IRISH
(4, 402)
CASS
(4, 402)
SHADOW
(4, 402)
BOND
(4, 402)
NAPE
(4, 402)
AMP
(4, 402)
FACE
(4, 402)
DEFOORE
(4, 402)
DURNIN
(4, 402)
CLIP
(4, 402)
COPEN
(4, 402)
CRUTCH
(4, 402)
REBEL
(4, 402)
TAOS
(4, 402)
COACH
(4, 402)
HEIST
(4, 402)
HOLSTER
(4, 402)
KONG
(4, 402)
LITTLE
(4, 402)
DUKE
(4, 402)
PITBULL
(4, 402)
HEAT
(4, 402)
FLIP
(4, 403)
WORM
(4, 403)
SKIDS
(4, 403)
SHINER
(4, 403)
APE
(4, 403)
BUSTER
(4, 403)
HATTRICK
(4, 403)
JACKAL
(4, 403)
HOOPS
(4, 403)
IRISH
(4, 403)
CASS
(4, 403)
SHADOW
(4, 403)
BOND
(4, 403)
NAPE
(4, 403)
AMP
(4, 403)
FACE
(4, 403)
DEFOORE
(4, 403)
DURNIN
(4, 403)
CLIP
(4, 403)
COPEN
(4, 403)
CRUTCH
(4, 403)
REBEL
(4, 403)
TAOS
(4, 403)
COACH
(4, 403)
HEIST
(4, 403)
HOLSTER
(4, 403)
KONG
(4

SKIDS
(7, 701)
SHINER
(7, 701)
APE
(7, 701)
BUSTER
(7, 701)
HATTRICK
(7, 701)
JACKAL
(7, 701)
HOOPS
(7, 701)
IRISH
(7, 701)
CASS
(7, 701)
SHADOW
(7, 701)
BOND
(7, 701)
NAPE
(7, 701)
AMP
(7, 701)
FACE
(7, 701)
DEFOORE
(7, 701)
DURNIN
(7, 701)
CLIP
(7, 701)
COPEN
(7, 701)
CRUTCH
(7, 701)
REBEL
(7, 701)
TAOS
(7, 701)
COACH
(7, 701)
HEIST
(7, 701)
HOLSTER
(7, 701)
KONG
(7, 701)
LITTLE
(7, 701)
DUKE
(7, 701)
PITBULL
(7, 701)
HEAT
(7, 701)
FLIP
(7, 702)
WORM
(7, 702)
SKIDS
(7, 702)
SHINER
(7, 702)
APE
(7, 702)
BUSTER
(7, 702)
HATTRICK
(7, 702)
JACKAL
(7, 702)
HOOPS
(7, 702)
IRISH
(7, 702)
CASS
(7, 702)
SHADOW
(7, 702)
BOND
(7, 702)
NAPE
(7, 702)
AMP
(7, 702)
FACE
(7, 702)
DEFOORE
(7, 702)
DURNIN
(7, 702)
CLIP
(7, 702)
COPEN
(7, 702)
CRUTCH
(7, 702)
REBEL
(7, 702)
TAOS
(7, 702)
COACH
(7, 702)
HEIST
(7, 702)
HOLSTER
(7, 702)
KONG
(7, 702)
LITTLE
(7, 702)
DUKE
(7, 702)
PITBULL
(7, 702)
HEAT
(7, 702)
FLIP
(7, 703)
WORM
(7, 703)
SKIDS
(7, 703)
SHINER
(7, 703)
APE
(7, 703)
BUSTER
(7, 703)
HATTRICK

(9, 908)
WORM
(9, 908)
SKIDS
(9, 908)
SHINER
(9, 908)
APE
(9, 908)
BUSTER
(9, 908)
HATTRICK
(9, 908)
JACKAL
(9, 908)
HOOPS
(9, 908)
IRISH
(9, 908)
CASS
(9, 908)
SHADOW
(9, 908)
BOND
(9, 908)
NAPE
(9, 908)
AMP
(9, 908)
FACE
(9, 908)
DEFOORE
(9, 908)
DURNIN
(9, 908)
CLIP
(9, 908)
COPEN
(9, 908)
CRUTCH
(9, 908)
REBEL
(9, 908)
TAOS
(9, 908)
COACH
(9, 908)
HEIST
(9, 908)
HOLSTER
(9, 908)
KONG
(9, 908)
LITTLE
(9, 908)
DUKE
(9, 908)
PITBULL
(9, 908)
HEAT
(9, 908)
FLIP


In [15]:
for line in lines.index:
    print(sum(pilot_status[line,x]*PQ[x] for x in pilots.index))
    print(LR[line])

(((((((((((((((((((((((((((((((0+((v1)*(p1)))+((v2)*(p2)))+((v3)*(p3)))+((v4)*(p4)))+((v5)*(p5)))+((v6)*(p6)))+((v7)*(p7)))+((v8)*(p8)))+((v9)*(p9)))+((v10)*(p10)))+((v11)*(p11)))+((v12)*(p12)))+((v13)*(p13)))+((v14)*(p14)))+((v15)*(p15)))+((v16)*(p16)))+((v17)*(p17)))+((v18)*(p18)))+((v19)*(p19)))+((v20)*(p20)))+((v21)*(p21)))+((v22)*(p22)))+((v23)*(p23)))+((v24)*(p24)))+((v25)*(p25)))+((v26)*(p26)))+((v27)*(p27)))+((v28)*(p28)))+((v29)*(p29)))+((v30)*(p30)))+((v31)*(p31)))
p32
(((((((((((((((((((((((((((((((0+((v32)*(p1)))+((v33)*(p2)))+((v34)*(p3)))+((v35)*(p4)))+((v36)*(p5)))+((v37)*(p6)))+((v38)*(p7)))+((v39)*(p8)))+((v40)*(p9)))+((v41)*(p10)))+((v42)*(p11)))+((v43)*(p12)))+((v44)*(p13)))+((v45)*(p14)))+((v46)*(p15)))+((v47)*(p16)))+((v48)*(p17)))+((v49)*(p18)))+((v50)*(p19)))+((v51)*(p20)))+((v52)*(p21)))+((v53)*(p22)))+((v54)*(p23)))+((v55)*(p24)))+((v56)*(p25)))+((v57)*(p26)))+((v58)*(p27)))+((v59)*(p28)))+((v60)*(p29)))+((v61)*(p30)))+((v62)*(p31)))
p33
(((((((((((((((((((((((

We build up our constraints

In [16]:
for line in lines.index:
    m.Equation( sum(pilot_status[line,x]*PQ[x] for x in pilots.index) >= LR[line])

# for line in lines.index:
    m.Equations( sum(pilot_status[(line,x)] for x in pilots.index) == 1) #only one pilot flying each line

In [17]:
for pilot in pilots.index:
    for go1,line1 in lines.index:
        m.Equation( sum(pilot_status[((go1,line2),pilot)] for go2,line2 in lines.index if go2==go1) <= 1 ) #pilot can only fly once each go

In [18]:
for go,line in lines.index:
    for pilot in pilots.index:
        if(pilots.loc[pilot][go-1]):
            m.Equation( pilot_status[((go,line),pilot)] == 0 ) #pilot is unavailable for that line

In [19]:
pilots

,1,2,3,4,5,6,7,8,9
PILOT,,,,,,,,,
WORM,0,0,0,0,0,0,0,0,0
SKIDS,0,0,0,0,0,0,0,0,0
SHINER,0,0,0,0,0,0,0,0,0
APE,0,0,0,0,0,0,0,0,0
BUSTER,0,0,0,0,0,0,0,0,0
HATTRICK,0,0,0,0,0,0,0,0,0
JACKAL,0,0,0,0,0,0,0,0,0
HOOPS,0,0,0,0,0,0,0,0,0
IRISH,0,0,0,0,0,0,0,0,0


In [20]:
m

We then solve the model

In [21]:
m.solve()

apm 99.22.36.105_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 @error: Inequality Definition
 invalid inequalities: z > x < y
 minimize<generatorobject<genexpr>at0x7fbd2581b650>
 STOPPING . . .


Exception:  @error: Inequality Definition
 invalid inequalities: z > x < y
 minimize<generatorobject<genexpr>at0x7fbd2581b650>
 STOPPING . . .


Now we'll display the solution found by the GEKKO solver

In [ ]:
pilot_status[((2,203),'SHINER')].value

In [ ]:
output = []
for Line, PILOT in pilot_status:
    if (pilot_status[(Line,PILOT)].value):
        var_output = {
            'Line': Line,
            'PILOT': PILOT,
        }
        output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Line'])
output_df.set_index(['Line', 'PILOT'], inplace=True)
output_df

In [ ]:
# Print our objective function value (Total Costs)
print (model.objective)